In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [64]:
def get_numpy_data(dataframe,features,output):
    feature_matrix = dataframe.loc[:,features]
    output = np.array(dataframe.loc[:,output])
    return(np.array(feature_matrix),output)
def normalize_features(features):
    norms = np.linalg.norm(features, axis=0)
    normalized_features = np.divide(features, norms)
    return normalized_features, norms

In [65]:
dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 'grade':int, 'yr_renovated':int, 'price':float, 'bedrooms':float,  'long':float, 'sqft_lot15':float, 'sqft_living':float, 'floors':float, 'condition':int, 'lat':float, 'sqft_basement':int, 'yr_built':int, 'sqft_lot':int, 'view':int}
dtype_list = [*dtype_dict.keys()]
data = pd.read_csv('Desktop/kc_house_data_small.csv',dtype=dtype_dict)
training = pd.read_csv('Desktop/kc_house_data_small_train.csv',dtype=dtype_dict)
test = pd.read_csv('Desktop/kc_house_data_small_test.csv',dtype=dtype_dict)
val = pd.read_csv('Desktop/kc_house_data_validation.csv',dtype=dtype_dict)
features = dtype_list

In [356]:
features_train,output_train= get_numpy_data(training,features,'price')
features_test,output_test = get_numpy_data(test,features,'price')
features_valid,output_valid = get_numpy_data(val,features,'price')
features_train, norms = normalize_features(features_train)
features_test = features_test / norms
features_valid = features_valid / norms

In [67]:
print( len((features_test[0])))
print(features_train[9])

18
[ 0.00602491  0.          0.0096309   0.01195898  0.01390535  0.
  0.01396435  0.01163464 -0.01346251  0.00156612  0.0083488   0.01279425
  0.01938684  0.01346821  0.          0.01302544  0.00050756  0.        ]


In [77]:
np.sqrt(np.sum((features_train[9] - features_test[0])**2))


0.059724654206600405

In [90]:
distances=[]
for i in range(10):
    distances.append(np.sqrt(np.sum((features_train[i] - features_test[0])**2))
)


In [91]:
index = np.argsort(np.array(distances))
distances[index[0]]
index


array([8, 3, 6, 7, 4, 9, 5, 0, 2, 1], dtype=int64)

In [97]:
diff =  features_train -features_test[0]

In [100]:
diff[-1].sum()

-0.09950545393080064

In [103]:
np.sum(diff**2, axis=1)[15]

0.0033580271173498403

In [104]:
np.sum(diff[15]**2)

0.0033580271173498403

In [107]:
distances = np.sum(diff**2, axis=1) ** 0.5

In [108]:
distances[100]

0.023901797554511096

In [123]:
def compute_distances(features_instances, features_query):
    diff =  features_instances -features_query
    distances = np.sqrt(np.sum(diff**2, axis=1))
    return distances

In [369]:
distances2 = compute_distances(features_train ,features_test[2]) 
index2 = np.argmin(np.array(distances2))
#distances2[index2[0]]
index2

1149

In [370]:
distances2.argsort()
output_train[382]

249000.0

In [333]:

def k_nearest_neighbors(k, features_train, query_vector):
    diff = features_train - query_vector
    distances = np.sqrt(np.sum(diff**2, axis=1))
    indices = distances.argsort()
    return indices[0:k]
def predicted_k_nearest_neighbors(k, features, output, query):
    indices = k_nearest_neighbors(k, features, query)
    prices = output[indices]
    avg = np.average(prices)
    return avg
def predict_output(k, features, output, query):
    predection = []
    #query = query.reshape(-1,1)
    for i in range(query.shape[0]):
        indices = k_nearest_neighbors(k, features, query[i])
        prices = output[indices]
        predection.append(np.average(prices))
    return predection

In [330]:
d=compute_distances(features_train[0:10],features_test[2])
print(d)
np.argsort(d)[::-1]
np.sort(d)[::-1]
features_test[2].shape

[0.02006157 0.06864228 0.02231441 0.01329609 0.01996616 0.01937183
 0.00697015 0.02976025 0.01627503 0.02235702]


(18,)

In [372]:
output_train[1149]
predicted_k_nearest_neighbors(4,features_train,output_train,features_test[2])
#predict_output(10,features_train,output_train,features_test[:10])
k_nearest_neighbors(4,features_train,features_test[2])
#np.mean(output_train[[1149, 3142, 4087, 2024]])


array([1149, 3142, 4087, 2024], dtype=int64)

In [237]:
        #dist = compute_distances(features_train[1] ,features_train[2])
diff =  features_train[1] -features_train[2]
dist = np.sqrt(np.sum(diff**2))

In [238]:
dist

0.07060059908797389

In [275]:
1883450.0/4

470862.5

In [342]:
def computeRSS(y_true,y_pred):
    return np.sum((y_true-y_pred)**2)

In [357]:
k = np.arange(1,16)
rss = []
for i in range(1,16):
    ouptput_pred = predict_output(i,features_train,output_train,features_valid)
    rss.append(computeRSS(output_valid,ouptput_pred))

In [358]:
min(rss)

12855658480182.889

In [359]:
rss

[17204161325646.0,
 13384777837645.75,
 12855658480182.889,
 13023884476897.188,
 13650723926462.918,
 14610032702990.252,
 14314185389596.164,
 14483166750826.61,
 14705658938667.062,
 15081873913409.79,
 15098381967074.0,
 15797225714549.582,
 16414473841793.51,
 17075754766112.77,
 17637717323868.32]

In [360]:
ouptput_pred = predict_output(3,features_train,output_train,features_test)
computeRSS(output_test,ouptput_pred)

39035062178637.125

In [351]:
39035062178637.125

3.9035062178637125